<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 2.1.1
# *Data Wrangling and Munging with Pandas*

## Part 1: Wrangling Data

The term "data wrangling" is analogous to capturing wild horses and getting them into a fenced area; the horses are data and the fencing is your computer. The more common data wrangling tasks include:

- reading flat files
- reading Excel files
- downloading from web pages
  - csv
  - html
  - json

In [3]:
import numpy as np
import pandas as pd

*It is good practice to display the library version numbers for future reference:*

In [ ]:
print('Numpy: ', np.__version__)
print('Pandas: ', pd.__version__)

### CSV Files

Below are three attempts to load the file "bikeshare.csv" into a DataFrame named `bikes`. Why are they wrong?

In [ ]:
# wrong:
bikes = pd.read_table('bikeshare.csv', header = None)
print(bikes.head())
print()

# wrong:
bikes = pd.read_table('bikeshare.csv', header = 1)
print(bikes.head())
print()

# wrong:
bikes = pd.read_table('bikeshare.csv', header = 0)
print(bikes.head())

ANSWER:

Load the file "bikeshare.csv" into a DataFrame named `bikes`, and confirm that it was loaded properly:

In [ ]:
#ANSWER:


In [35]:
bikes = pd.read_table('../DATA/bikeshare.csv', header=0, sep="-")
print(bikes.head())

         instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
1,2011 1      01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0,3,13,16                                                                   
2,2011 1       01,1,0,1,1,0,6,0,1,0.22,0.2727,0.8,0,8,32,40                                                                   
3,2011 1       01,1,0,1,2,0,6,0,1,0.22,0.2727,0.8,0,5,27,32                                                                   
4,2011 1      01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0,3,10,13                                                                   
5,2011 1        01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0,0,1,1                                                                   


Note that we could have used `read.csv()` above. When is `read_table()` necessary?

?:
ANSWER: When `sep` is not the comma character, or we need fine control that `read.csv()` does not provide.

Flat files can be full of surprises. Here are some issues to watch out for:

- separator character is something other than the comma
  - ";", "|", and tab are popular
- newline character is something other than what the O/S expects
  - Tip: Don't hard-code the character codes for carriage returns, linefeeds, etc. Use Python's built-in representation instead (e.g. Python translates "\n" to the newline character and "\t" to the tab character on any O/S).
- truncated lines
  - if there are empty fields at the end of a line it is possible that their separators will be missing, resulting in a "jagged" file
- embedded commas or quotes
  - a free-text field containing embedded commas may split into separate fields on input
  - a free-text field containing embedded quotes may not parse correctly
- unescaped characters
  - the "\" character indicates a control code to Python, which will break the I/O
    - e.g. the substring "\u0123" will be interpreted as Unicode(0123) -- which may not be what the file creator intended
  - these may need to be fixed by loading whole strings and then parsing into a new data frame
  
Tip: Most issues can be dealt with by correctly specifying the parameters of the function you use to load the file. Read the doco before reading the data!

### Reading Excel Files

In [28]:
from pandas import ExcelFile  # Nb. Need to install xlrd from conda (it does not automatically install with pandas)
import xlrd

In [70]:
df = pd.read_excel('../DATA/Iris.xlsx', sheet_name ='Data')
df

FileNotFoundError: [Errno 2] No such file or directory: '../DATA/Iris.xlsx'

It is usually better to load data correctly than to meddle with the source file or load it 'warts and all' and then try to parse it in code. The Pandas functions for reading files have parameters that provide the control we need. For example, we could make multiple calls to `read_excel()`, using combinations of the `header`, `usecols`, `skiprows`, `nrows`, and `skipfooter` parameters to load one table at a time from a spreadsheet with multiple tables.

Load the above file without the unwanted columns:

In [58]:
#ANSWER
df = pd.read_excel('../DATA/Iris.xlsx')
df

FileNotFoundError: [Errno 2] No such file or directory: '../DATA/Iris.xlsx'

### Importing Data Directly from the Web

We usually want to store a local copy of a data file that we download from the Web, but when data retention is not a priority it is convenient to download the data directly into our running Python environment.

#### Importing Text Files from the Web

The web is the 'wild west' of data formats. However, we can usually expect good behaviour from files that are automatically generated by a service, such as the earthquake report:

In [37]:
df = pd.read_csv('https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/2.5_hour.csv')
df.head()

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2023-12-14T02:25:51.480Z,19.383667,-155.277333,1.44,2.94,ml,38,67,NaN,0.11,...,2023-12-14T02:46:15.123Z,"7 km SW of Volcano, Hawaii",earthquake,0.16,0.13,0.256393,24,reviewed,hv,hv
1,2023-12-14T02:25:43.020Z,19.385334,-155.280502,0.39,2.91,ml,7,160,NaN,0.15,...,2023-12-14T02:31:16.170Z,"8 km SW of Volcano, Hawaii",earthquake,0.35,0.45,2.090000,21,automatic,hv,hv
2,2023-12-14T02:25:28.520Z,19.285667,-155.385666,1.79,2.87,ml,26,129,NaN,0.21,...,2023-12-14T02:30:59.590Z,"13 km NE of Pāhala, Hawaii",earthquake,0.43,0.78,2.200000,21,automatic,hv,hv
3,2023-12-14T02:17:14.170Z,19.393333,-155.272507,0.99,2.64,ml,36,41,NaN,0.24,...,2023-12-14T02:22:45.340Z,"6 km SW of Volcano, Hawaii",earthquake,0.24,0.20,2.010000,14,automatic,hv,hv


#### Importing HTML Files from the Web

Working with unstructured HTML files relies heavily on library functions. This one, however, is well-structured:

In [60]:
url = 'https://www.ccra.com/airport-codes/'

df = pd.read_html(url)
df

[                City      Country Code
 0            Aalborg      Denmark  AAL
 1           Aalesund       Norway  AES
 2             Aarhus      Denmark  AAR
 3     Abbotsford, BC       Canada  YXX
 4     Abbotsford, BC       Canada  YXX
 ...              ...          ...  ...
 1574        Zanzibar     Tanzania  ZNZ
 1575        Zaragoza        Spain  ZAZ
 1576       Zhengzhou        China  CGO
 1577        Zhoushan        China  HSN
 1578          Zurich  Switzerland  ZRH
 
 [1579 rows x 3 columns]]

In [39]:
df[0]

,City,Country,Code
0,Aalborg,Denmark,AAL
1,Aalesund,Norway,AES
2,Aarhus,Denmark,AAR
3,"Abbotsford, BC",Canada,YXX
4,"Abbotsford, BC",Canada,YXX
...,...,...,...
1574,Zanzibar,Tanzania,ZNZ
1575,Zaragoza,Spain,ZAZ
1576,Zhengzhou,China,CGO
1577,Zhoushan,China,HSN


#### Importing XML Files from the Web

XML files are semi-structured, but you're at the mercy of the file creator. If every record has the same format it will be much easier, but practical applications often require a lot of custom code. Here are a few examples: https://pandas.pydata.org/docs/user_guide/io.html#io-read-xml

#### Importing JSON Files from the Web

Like XML, JSON files are semi-structured and may require work to capture the schema into a dataframe. Here is a simple example:

In [57]:
url = 'https://microsoftedge.github.io/Demos/json-dummy-data/64KB.json'

# Load the first sheet of the JSON file into a data frame
df = pd.read_json(url, orient = 'columns')
df.head()

,name,language,id,bio,version
0,Adeel Solangi,Sindhi,V59OF92YF627HFY0,Donec lobortis eleifend condimentum. Cras dict...,6.10
1,Afzal Ghaffar,Sindhi,ENTOCR13RSCLZ6KU,"Aliquam sollicitudin ante ligula, eget malesua...",1.88
2,Aamir Solangi,Sindhi,IAKPO3R4761JDRVG,Vestibulum pharetra libero et velit gravida eu...,7.27
3,Abla Dilmurat,Uyghur,5ZVOEPMJUI4MB4EN,Donec lobortis eleifend condimentum. Morbi ac ...,2.53
4,Adil Eli,Uyghur,6VTI8X6LL0MMPJCC,"Vivamus id faucibus velit, id posuere leo. Mor...",6.49


## Part 2: Data Munging

Data munging is manipulating data to get it into a form that we can start running analyses on (which usually means getting the data into a DataFrame). Before we get to this stage, we may need to remove headers or footers, transpose columns to rows, split wide data tables into long ones, and so on. (Nb. Excel files can be particularly troublesome, because users can format their data in mixed, complex shapes.) Essentially, we need to follow Hadley Wickham's guidelines for tidy datasets (http://vita.had.co.nz/papers/tidy-data.html):

The end goal of the cleaning data process:

- each variable should be in one column
- each observation should comprise one row
- each type of observational unit should form one table
- include key columns for linking multiple tables
- the top row contains (sensible) variable names
- in general, save data as one file per table


### Dataset Morphology

Once we have our dataset in a DataFrame (or Series, if our data is only 1-dimensional), we can start examining its size and content.

How many rows and columns are in `bikes`?

In [41]:
#ANSWER
bikes.shape

(17379, 1)

What are the column names in `bikes`?

In [42]:
#ANSWER
bikes.columns

Index(['instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt'], dtype='object')

What are the data types of these columns?

In [43]:
#ANSWER
bikes.dtypes

instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt    object
dtype: object

What is the (row) index for this DataFrame?

In [44]:
#ANSWER
bikes.index

MultiIndex([(    '1,2011',  1),
            (    '2,2011',  1),
            (    '3,2011',  1),
            (    '4,2011',  1),
            (    '5,2011',  1),
            (    '6,2011',  1),
            (    '7,2011',  1),
            (    '8,2011',  1),
            (    '9,2011',  1),
            (   '10,2011',  1),
            ...
            ('17370,2012', 12),
            ('17371,2012', 12),
            ('17372,2012', 12),
            ('17373,2012', 12),
            ('17374,2012', 12),
            ('17375,2012', 12),
            ('17376,2012', 12),
            ('17377,2012', 12),
            ('17378,2012', 12),
            ('17379,2012', 12)],
           length=17379)

https://www.dataquest.io/blog/python-json-tutorial/

## Slicing and Dicing

It is often preferable to refer to DataFrame columns by name, but there is more than one way to do this.
Do `bikes['season']` and `bikes[['season']]` give the same object? Demonstrate:

In [61]:
#ANSWER
print(bikes['season'].head())
print(type(bikes[['season']]))

KeyError: 'season'

How would we use object notation to show the first 4 rows of `atemp`?

In [54]:
#ANSWER
print(bikes.atemp[:4])

AttributeError: 'DataFrame' object has no attribute 'atemp'

In [53]:
print(bikes.columns)


Index(['instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt'], dtype='object')


Algorithms that loop over multiple columns often access DataFrame columns by index. However, none of the following work (try them out by uncommenting / removing the "#E: " ):

In [62]:
bikes[[0]]
#E: bikes[0]
#E: bikes[0,0]
#E: bikes[[0,0]]

KeyError: "None of [Index([0], dtype='int32')] are in the [columns]"

In [63]:
bikes[0]
bikes[0,0]
bikes[[0,0]]

KeyError: 0

What is the correct way to access the 1st row of the DataFrame by its index?

In [66]:
#ANSWER
bikes[:1]


,,"instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt"
"1,2011",1,"01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0,3,13,16"


In [67]:
bikes[0:1]

,,"instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt"
"1,2011",1,"01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0,3,13,16"


What is the correct way to access the 2nd column of the DataFrame by its index?

In [68]:
#ANSWER
bikes.iloc[:, 1]

IndexError: single positional indexer is out-of-bounds

## Handling Missing Values

What is the Pandas `isnull` function for?

?
ANSWER:

We can apply `isnull` to the `bikes` DataFrame to show the result for every element:

In [74]:
bikes.isnull().head()

,,"instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt"
"1,2011",1,False
"2,2011",1,False
"3,2011",1,False
"4,2011",1,False
"5,2011",1,False


However, we usually start at a higher level. How many nulls are in `bikes` altogether?

In [75]:
bikes['windspeed'].isnull().sum()

KeyError: 'windspeed'

If this result were nonzero we would next want to find out which columns contained nulls. How can this be done in one line of code?

In [76]:
#ANSWER
bikes.isnull().sum()

instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt    0
dtype: int64

What is the Numpy object `nan` used for? (Write a descriptive answer.)

#ANSWER 
It shows missing or undefined values in datasets

Write (and verify) a function that performs scalar division with built-in handling of the edge case (i.e. return a value instead of just trapping the error):

In [79]:
#ANSWER
def divide(numerator, denominator, default_value=None):
    
    if denominator == 0:
        return default_value
    else:
        return numerator / denominator

print(divide(10,2))

5.0


Apply the Pandas `isna` function to the following data objects:

In [83]:
x = 2.3
y = np.nan
print(x, y)

2.3 nan


In [85]:
#ANSWER
print(pd.isna(x), pd.isna(y))

False True


In [86]:
array = np.array([[1, np.nan, 3], [4, 5, np.nan]])
print(array)

[[ 1. nan  3.]
 [ 4.  5. nan]]


In [87]:
#ANSWER
print(pd.isna(array))

[[False  True False]
 [False False  True]]


How is the pandas I/O parameter `na_values` used?

? ANSWER: It allows the customisation of representing missing values (NaN) as items such as NA or missing

## Data Profiling

### Counts

When there are categorical variables in a dataset we will want to know how many possible values there are in each column. (Nb. If the dataset is a sample of a larger one, our sample may not capture all possible values of every categorical.)

How many (different) seasons are in `bikes`?

In [ ]:
#ANSWER


In [88]:
bikes['season'].value_counts()

KeyError: 'season'

### Ranges

Print the range of the `instant`, `dteday`, and `windspeed` columns:

In [89]:
#ANSWER
print('instant:', bikes['instant'].min(), 'to', bikes['instant'].max())
print('dteday:', bikes['dteday'].min(), 'to', bikes['dteday'].max())
print('windspeed:', bikes['windspeed'].min(), 'to', bikes['windspeed'].max())

KeyError: 'instant'

Compute and print the overall minimum and maximum of the numeric data columns:

In [90]:
bikes_min, bikes_max = (min(bikes.min()), max(bikes.max()))
bikes_min, bikes_max

('01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0,3,13,16',
 '31,4,1,10,9,0,3,1,1,0.34,0.3333,0.76,0.1343,17,280,297')

In [92]:
bikes_min, bikes_max = (bikes.min().min(), bikes.max().max())

print("Overall Minimum Value:", bikes_min)
print("Overall Maximum Value:", bikes_max)

Overall Minimum Value: 01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0,3,13,16
Overall Maximum Value: 31,4,1,10,9,0,3,1,1,0.34,0.3333,0.76,0.1343,17,280,297


### Quantiles

Pandas makes computing quantiles easy. This is how to get the median of a Series:

In [93]:
bikes['atemp'].quantile(0.5)

KeyError: 'atemp'

Of course, the `quantiles` method can take a tuple as its argument. Compute the 10th, 25th, 50th, 75th, and 90th percentiles in one line of code:

In [94]:
#ANSWER
bikes['atemp'].quantile((0.1, 0.25, 0.5, 0.75, 0.9))

KeyError: 'atemp'

### Cuts

Sometimes we want to split the sample not by the quantiles of the distribution but by the range of the data. Let's take a closer look at `atemp`:

In [95]:
type(bikes['atemp'])

KeyError: 'atemp'

In [96]:
bikes.sample(5)

,,"instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt"
"6895,2011",10,"19,4,0,10,21,0,3,1,2,0.58,0.5455,0.94,0.1343,6..."
"5915,2011",9,"08,3,0,9,23,0,4,1,3,0.62,0.5152,1,0.0896,0,26,26"
"6162,2011",9,"19,3,0,9,7,0,1,1,2,0.5,0.4848,0.82,0.2239,20,3..."
"1282,2011",2,"26,1,0,2,13,0,6,0,2,0.34,0.3182,0.42,0.2537,38..."
"266,2011",1,"12,1,0,1,12,0,3,1,1,0.2,0.1515,0.47,0.5821,3,5..."


Suppose we decide to sort these values into 4 bins of equal width, but we want to apply the resulting groups to the entire DataFrame. Basically, we need to add a row label that indicates which bin each sample belongs in. Let's call this label "atemp_level", and use the `cut` method to populate it:

In [97]:
atemp_level = pd.cut(bikes['atemp'], bins = 4)

KeyError: 'atemp'

What is `atemp_level`?

In [98]:
#ANSWER
print(type(atemp_level))

NameError: name 'atemp_level' is not defined

Here is a random sample of `atemp_level`:

In [99]:
atemp_level.sample(5)

NameError: name 'atemp_level' is not defined

So, by default, `cut` produces labels that indicate the bin boundaries for each element in the series it was applied to. Usually, we will specify labels that are appropriate to the discretisation we are applying:

In [100]:
atemp_level = pd.cut(bikes['atemp'], bins = 4, labels = ["cool", "mild", "warm", "hot"])
atemp_level.sample(5)

KeyError: 'atemp'

Incorporate the new `atemp_level` column into the `bikes` DataFrame and use it to count the number of "mild" `atemp` entries in `season` 2:

In [101]:
#ANSWER
bikes['atemp_level'] = atemp_level
bikes[(bikes.atemp_level == 'mild') & (bikes.season == 2)].season.count()

NameError: name 'atemp_level' is not defined

*Nb. The `atemp_level` variable we created is what the R language calls a "factor". Pandas has introduced a new data type called "category" that is similar to R's factors.*

# Synthetic Data

Sometimes we may want to generate test data, or we may need to initialise a series, matrix, or data frame for input to an algorithm. Numpy has several methods we can use for this.

Execute the following, then check the shape and content of each variable:

In [102]:
# Creating arrays with initial values
a = np.zeros((3))
b = np.ones((1,3))
c = np.random.randint(1,10,(2,3,4))   # randint(low, high, size)
d = np.arange(4)
e = np.array( [[1,2,3,4], [5,6,7,8]] )

## Load Data

Load rock.csv and clean the dataset.

In [104]:
# ANSWER
rock_csv = '../DATA/rock.csv'
rock = pd.read_csv(rock_csv)

## Check Column Names

Check column names and clean.

In [105]:
# ANSWER
rock.columns

Index(['Song Clean', 'ARTIST CLEAN', 'Release Year', 'COMBINED', 'First?',
       'Year?', 'PlayCount', 'F*G'],
      dtype='object')

## Replace Null Values With 0

Check 'release_year' column whether this column has any null values or not. Replace null values with 0.

In [106]:
# ANSWER
rock.isnull().sum()

Song Clean        0
ARTIST CLEAN      0
Release Year    577
COMBINED          0
First?            0
Year?             0
PlayCount         0
F*G               0
dtype: int64

## Check Datatypes of Dataset

Check datatypes of the dataset. Is there any column which should be int instead of object? Fix the column.

In [107]:
# ANSWER
rock.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2230 entries, 0 to 2229
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Song Clean    2230 non-null   object
 1   ARTIST CLEAN  2230 non-null   object
 2   Release Year  1653 non-null   object
 3   COMBINED      2230 non-null   object
 4   First?        2230 non-null   int64 
 5   Year?         2230 non-null   int64 
 6   PlayCount     2230 non-null   int64 
 7   F*G           2230 non-null   int64 
dtypes: int64(4), object(4)
memory usage: 139.5+ KB


## Check Min, Max of Each Column

Is there any illogical value in any column? How can we fix that?

In [110]:
# ANSWER
def check_min_max(df):
    # Check min, max of each column
    print(df.describe().T[['min', 'max']])
    
check_min_max(rock)

           min    max
First?     1.0    1.0
Year?      0.0    1.0
PlayCount  0.0  142.0
F*G        0.0  142.0


## Write Some Functions

### Write a function that will take a row of a DataFrame and print out the song, artist, and whether or not the release date is < 1970

In [112]:
# ANSWER
rock.dtypes

Song Clean      object
ARTIST CLEAN    object
Release Year    object
COMBINED        object
First?           int64
Year?            int64
PlayCount        int64
F*G              int64
dtype: object

In [114]:
def check_song(row):
    print('Song: ', row['song_clean'])
    print('Artist: ', row['artist_clean'])
    print('Released before 1970: ', row['release_year'] < 1970)
    print('\n')

In [115]:
rock.iloc[0]

Song Clean                     Caught Up in You
ARTIST CLEAN                        .38 Special
Release Year                               1982
COMBINED        Caught Up in You by .38 Special
First?                                        1
Year?                                         1
PlayCount                                    82
F*G                                          82
Name: 0, dtype: object

### Write a function that converts a column in a DataFrame to a numeric type and otherwise replaces entries with np.nan

In [118]:
# ANSWER

import pandas as pd
import numpy as np

def convert_to_numeric(df, column_name):
    df_copy = df.copy()
    df_copy[column_name] = pd.to_numeric(df_copy[column_name], errors='coerce')
    return df_copy

### Apply this last function to your dataset

In [120]:
# ANSWER

def convert_to_numeric(series):
    return pd.to_numeric(series, errors='coerce')

columns_to_convert = ['release_year', 'first', 'year', 'playcount', 'fg']

rock[columns_to_convert] = rock[columns_to_convert].apply(convert_to_numeric)

KeyError: "None of [Index(['release_year', 'first', 'year', 'playcount', 'fg'], dtype='object')] are in the [columns]"

### 'Describe' the new DataFrame

In [124]:
# ANSWER
rock.describe().T

,count,mean,std,min,25%,50%,75%,max
First?,2230.0,1.000000,0.000000,1.0,1.0,1.0,1.0,1.0
Year?,2230.0,0.741256,0.438043,0.0,0.0,1.0,1.0,1.0
PlayCount,2230.0,16.872646,25.302972,0.0,1.0,4.0,21.0,142.0
F*G,2230.0,15.048430,25.288366,0.0,0.0,3.0,18.0,142.0


>




---



---



> > > > > > > > > © 2023 Institute of Data


---



---



